# IBM-Coursera Data Science Capstone: Canadian Cities Comparison

This is the final submission of the Data Science capstone project.

## 1) Obtain the required Montreal and Vancouver Neighborhood Data. 

1.1. Download the required libraries.

In [1]:
import pandas as pd 
import requests
import numpy as np
from pandas.io.json import json_normalize
import json
from geopy.geocoders import Nominatim
import wget
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


1.2. Read the required Montreal and vancouver neighborhood data.

I created my own datasets with Montreal's and Vancouver's neighborhood, borough, zip code and geographical coordinates data. I mainly used these web pages https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H and https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V

In [2]:
mdf = pd.read_csv('/Users/mjolnir/Desktop/Organizar/Coursera_Capstone/Coursera_Capstone/Montreal_data.csv',encoding = 'unicode_escape')
vdf = pd.read_csv('/Users/mjolnir/Desktop/Organizar/Coursera_Capstone/Coursera_Capstone/Vancouver_data.csv',encoding = 'unicode_escape')

## 2) Visual Inspection of Data

2.1. Check the check the data stored in each city data and data frame's dimensions.

In [4]:
Montreal_df = mdf[mdf.Borough != 'Not assigned'].reset_index(drop = True)
print(Montreal_df.shape)
Montreal_df.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,H1A,Riviere-des-Prairies-Pointe-aux-Trembles,Pointe-aux-Trembles,45.642333,-73.505306
1,H2A,Villeray-Saint-Michel-Parc-Extension,Saint-Michel East,45.563889,-73.609722
2,H3A,Ville-Marie,Downtown Montreal North,45.500000,-73.570000
3,H4A,Cote-des-Neiges-Notre-Dame-de-Grace,Notre-Dame-de-Grace Northeast,45.476750,-73.614320
4,H5A,Ville-Marie,Place Bonaventure,45.499400,-73.565100


In [6]:
Vancouver_df = vdf[vdf.Borough != 'Not assigned'].reset_index(drop = True)
print(Vancouver_df.shape)
Vancouver_df.head()

(66, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,V5K,Hastings-Sunrise,NorthÂ Hastings-Sunrise,49.2811,-123.0441
1,V5L,Grandview-Woodland,North Grandview Woodland,49.2767,-123.0695
2,V5M,Hastings-Sunrise,South Hastings-Sunrise,49.2811,-123.0441
3,V5M,Renfrew-Collingwood,North Renfrew-Collingwood,49.2445,-123.0422
4,V5N,Grandview-Woodland,South Grandview-Woodland,49.2767,-123.0695


## 3) Geographical Data Visualization and Partial Data Preparation.

3.1. Plot the location of the neighborhoods on the map.

In [7]:
lam = 45.50884
lonm = -73.58781
Montreal_map = folium.Map(location=[lam,lonm],zoom_start=11)

for lat, long, post, borough, neigh in zip(Montreal_df['Latitude'],Montreal_df['Longitude'], Montreal_df['PostalCode'],Montreal_df['Borough'],Montreal_df['Neighborhood']):
    label = "{} ({}): {}".format(borough,post,neigh)
    popup = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=popup,
    color='red',
    fill = True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(Montreal_map)

Montreal_map

In [8]:
lav = 49.2827
lonv = -123.1207
Vancouver_map = folium.Map(location=[lav,lonv],zoom_start=11.6)

for lat, long, post, borough, neigh in zip(Vancouver_df['Latitude'],Vancouver_df['Longitude'], Vancouver_df['PostalCode'],Vancouver_df['Borough'],Vancouver_df['Neighborhood']):
    label = "{} ({}): {}".format(borough,post,neigh)
    popup = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=popup,
    color='blue',
    fill = True,
    fill_color='blue',
    fill_opacity=0.7,
    parse_html=False).add_to(Vancouver_map)

Vancouver_map

3.2. The Foursquare's API has a limited number of allowed calls per day. In this capstone project it is explored the scenario where a student wants to have a better understanding of the neighborhoods close to McGill University and the University of British Columbia, based on the surrounding venues. Therefore, a limited number of neighborhoods were selected.

In [9]:
m = ['Ville-Marie','Le Plateau-Mont-Royal','Outremont','Le Sud-Ouest','Cote-des-Neiges-Notre-Dame-de-Grace']
Mtr_McGill_df = Montreal_df[Montreal_df['Borough'].isin(m)]
Mtr_McGill_df = Mtr_McGill_df.reset_index()
print(Mtr_McGill_df.shape)
Mtr_McGill_df.head()

(33, 6)


,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,H3A,Ville-Marie,Downtown Montreal North,45.50000,-73.57000
1,3,H4A,Cote-des-Neiges-Notre-Dame-de-Grace,Notre-Dame-de-Grace Northeast,45.47675,-73.61432
2,4,H5A,Ville-Marie,Place Bonaventure,45.49940,-73.56510
3,8,H3B,Ville-Marie,Downtown Montreal East,45.50000,-73.57000
4,9,H4B,Cote-des-Neiges-Notre-Dame-de-Grace,Notre-Dame-de-Grace Southwest,45.47675,-73.61432


In [10]:
v =['University Endowment Lands','West Point Grey','Dunbar-Southlands','Kitsilando','Arbutus Ridge','Kerrisdale','Shaughnessy','Fairview','South Cambie','Oakridge','Marpole']
Vanc_UBC_df = Vancouver_df[Vancouver_df['Borough'].isin(v)]
Vanc_UBC_df = Vanc_UBC_df.reset_index()
print(Vanc_UBC_df.shape)
Vanc_UBC_df.head()

(27, 6)


,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,15,V5W,Oakridge,NE Oakridge,49.2261,-123.1166
1,17,V5X,Oakridge,SE Oakridge,49.2261,-123.1166
2,18,V5X,Marpole,East Marpole,49.2107,-123.1302
3,22,V5Z,Fairview,East Fairview,49.2635,-123.1313
4,23,V5Z,South Cambie,South Cambie,49.2452,-123.1208


3.3. The geographical data of the new data frames are plotted on the map.

In [11]:
Montreal_map2 = folium.Map(location=[lam,lonm],zoom_start=13)

for lat, long, post, borough, neigh in zip(Mtr_McGill_df['Latitude'],Mtr_McGill_df['Longitude'], Mtr_McGill_df['PostalCode'],Mtr_McGill_df['Borough'],Mtr_McGill_df['Neighborhood']):
    label = "{} ({}): {}".format(borough,post,neigh)
    popup = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=popup,
    color='red',
    fill = True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(Montreal_map2)

Montreal_map2

In [12]:
Vancouver_map2 = folium.Map(location=[lav,lonv],zoom_start=12.3)

for lat, long, post, borough, neigh in zip(Vanc_UBC_df['Latitude'],Vanc_UBC_df['Longitude'],Vanc_UBC_df['PostalCode'],Vanc_UBC_df['Borough'],Vanc_UBC_df['Neighborhood']):
    label = "{} ({}): {}".format(borough,post,neigh)
    popup = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=popup,
    color='blue',
    fill = True,
    fill_color='blue',
    fill_opacity=0.7,
    parse_html=False).add_to(Vancouver_map2)

Vancouver_map2

## 4) Venue Data Obtention and Explotarion, and Further Data Preparation.

4.1. Using my credential I got access into the Foursquare API. I hid my credential, to be able to share it on Github

In [13]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = # Foursquare API version

4.2. Use the Foursquare's API to retrieve 50 venues within a radius of 1000 mts, for both cities.

In [14]:
radius = 1000
LIMIT = 50

venues_mtrl = []

for lat, long, post, borough, neighborhood in zip(Mtr_McGill_df['Latitude'], Mtr_McGill_df['Longitude'], Mtr_McGill_df['PostalCode'], Mtr_McGill_df['Borough'], Mtr_McGill_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues_mtrl.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [15]:
radius = 1000
LIMIT = 50

venues_van = []

for lat, lng, post, borough, neighborhood in zip(Vanc_UBC_df['Latitude'], Vanc_UBC_df['Longitude'], Vanc_UBC_df['PostalCode'], Vanc_UBC_df['Borough'], Vanc_UBC_df['Neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues_van.append((
            post, 
            borough,
            neighborhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

4.3. Create data frames of the found venues, check the dimensions of the data frames and get a look into the first data points.

In [16]:
venues_df_mtl = pd.DataFrame(venues_mtrl)
venues_df_mtl.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df_mtl.shape)
venues_df_mtl.head()

(1515, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,H3A,Ville-Marie,Downtown Montreal North,45.5,-73.57,Dominion Square Tavern,45.500405,-73.571636,Gastropub
1,H3A,Ville-Marie,Downtown Montreal North,45.5,-73.57,Cathédrale Marie-Reine-du-Monde,45.499614,-73.569367,Church
2,H3A,Ville-Marie,Downtown Montreal North,45.5,-73.57,Nacarat,45.500773,-73.568200,Cocktail Bar
3,H3A,Ville-Marie,Downtown Montreal North,45.5,-73.57,Café Humble Lion,45.501766,-73.570620,Coffee Shop
4,H3A,Ville-Marie,Downtown Montreal North,45.5,-73.57,Square Dorchester,45.499474,-73.570848,Park


In [17]:
venues_df_van = pd.DataFrame(venues_van)
venues_df_van.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df_van.shape)
venues_df_van.head()

(921, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,V5W,Oakridge,NE Oakridge,49.2261,-123.1166,The LEGO Store,49.231673,-123.118370,Toy / Game Store
1,V5W,Oakridge,NE Oakridge,49.2261,-123.1166,DAVIDsTEA,49.232683,-123.118074,Tea Room
2,V5W,Oakridge,NE Oakridge,49.2261,-123.1166,Langara Golf Course,49.223712,-123.112589,Golf Course
3,V5W,Oakridge,NE Oakridge,49.2261,-123.1166,Apple Oakridge Centre,49.232090,-123.119377,Electronics Store
4,V5W,Oakridge,NE Oakridge,49.2261,-123.1166,BC Liquor Store,49.234758,-123.116637,Liquor Store


4.4. Group the data within the data frames based on the postal code, borough and neighborhood and check how many venues were obtained by postal code.

In [34]:
venues_df_mtl.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

PostalCode  Borough                              Neighborhood                    
H2H         Le Plateau-Mont-Royal                Plateau Mont-Royal North            50
H2J         Le Plateau-Mont-Royal                Plateau Mont-Royal North Central    50
H2K         Ville-Marie                          Centre-Sud North                    50
H2L         Ville-Marie                          Centre-Sud South                    50
H2T         Le Plateau-Mont-Royal                Plateau Mont-Royal West             50
H2V         Outremont                            Outremont                           47
H2W         Le Plateau-Mont-Royal                Plateau Mont-Royal South Central    50
H2X         Le Plateau-Mont-Royal                Plateau Mont-Royal Southeast        50
H2Y         Ville-Marie                          Old Montreal                        50
H2Z         Ville-Marie                          Downtown Montreal Northeast         50
H3A         Ville-Marie               

In [19]:
venues_df_van.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

PostalCode  Borough                     Neighborhood                    
V5W         Oakridge                    NE Oakridge                         50
V5X         Marpole                     East Marpole                        50
            Oakridge                    SE Oakridge                         50
V5Z         Fairview                    East Fairview                       50
            South Cambie                South Cambie                        47
V6H         Fairview                    Granville Island                    50
                                        West Fairview                       50
            Shaughnessy                 NE Shaughnessy                      11
V6J         Shaughnessy                 NW Shaughnessy                      11
V6L         Arbutus Ridge               NW Arbutus Ridge                    30
            Dunbar-Southlands           NE Dunbar-Southlands                17
V6M         Arbutus Ridge               SE Arbutus Ridge  

4.5. Check how many unique categories of venues are stored within each data frame.

In [20]:
len(venues_df_mtl['VenueCategory'].unique())

162

In [21]:
len(venues_df_van['VenueCategory'].unique())

129

4.6. Perform one hot encoding to be able to analyze the venues per area.

In [22]:
# one hot encoding
Mtl_clust_onehot = pd.get_dummies(venues_df_mtl[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
Mtl_clust_onehot['PostalCode'] = venues_df_mtl['PostalCode'] 
Mtl_clust_onehot['Borough'] = venues_df_mtl['Borough'] 
Mtl_clust_onehot['Neighborhoods'] = venues_df_mtl['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(Mtl_clust_onehot.columns[-3:]) + list(Mtl_clust_onehot.columns[:-3])
Mtl_clust_onehot = Mtl_clust_onehot[fixed_columns]

print(Mtl_clust_onehot.shape)
Mtl_clust_onehot.head()

(1515, 165)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,H3A,Ville-Marie,Downtown Montreal North,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H3A,Ville-Marie,Downtown Montreal North,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,H3A,Ville-Marie,Downtown Montreal North,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H3A,Ville-Marie,Downtown Montreal North,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,H3A,Ville-Marie,Downtown Montreal North,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
Van_clust_onehot = pd.get_dummies(venues_df_van[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
Van_clust_onehot['PostalCode'] = venues_df_van['PostalCode'] 
Van_clust_onehot['Borough'] = venues_df_van['Borough'] 
Van_clust_onehot['Neighborhoods'] = venues_df_van['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(Van_clust_onehot.columns[-3:]) + list(Van_clust_onehot.columns[:-3])
Van_clust_onehot = Van_clust_onehot[fixed_columns]

print(Van_clust_onehot.shape)
Van_clust_onehot.head()

(921, 132)


,PostalCode,Borough,Neighborhoods,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Women's Store,Yoga Studio
0,V5W,Oakridge,NE Oakridge,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,V5W,Oakridge,NE Oakridge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,V5W,Oakridge,NE Oakridge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,V5W,Oakridge,NE Oakridge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,V5W,Oakridge,NE Oakridge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


4.7. Get the frequency of occurance of each category within an area.

In [24]:
Mtl_clust_venues_freq = Mtl_clust_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(Mtl_clust_venues_freq.shape)
Mtl_clust_venues_freq.head()

(33, 165)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store,Yoga Studio
0,H2H,Le Plateau-Mont-Royal,Plateau Mont-Royal North,0.00,0.02,0.0,0.0,0.0,0.0,0.00,...,0.0,0.02,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.02
1,H2J,Le Plateau-Mont-Royal,Plateau Mont-Royal North Central,0.00,0.02,0.0,0.0,0.0,0.0,0.00,...,0.0,0.02,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.02
2,H2K,Ville-Marie,Centre-Sud North,0.02,0.00,0.0,0.0,0.0,0.0,0.02,...,0.0,0.00,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.00
3,H2L,Ville-Marie,Centre-Sud South,0.02,0.00,0.0,0.0,0.0,0.0,0.02,...,0.0,0.00,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.00
4,H2T,Le Plateau-Mont-Royal,Plateau Mont-Royal West,0.00,0.02,0.0,0.0,0.0,0.0,0.00,...,0.0,0.02,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.02


In [25]:
Van_clust_venues_freq = Van_clust_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(Van_clust_venues_freq.shape)
Van_clust_venues_freq.head()

(27, 132)


,PostalCode,Borough,Neighborhoods,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Women's Store,Yoga Studio
0,V5W,Oakridge,NE Oakridge,0.00,0.00,0.00,0.000000,0.00,0.0,0.02,...,0.00,0.02,0.02,0.02,0.00,0.00,0.000000,0.00,0.04,0.0
1,V5X,Marpole,East Marpole,0.02,0.00,0.00,0.000000,0.00,0.0,0.00,...,0.02,0.00,0.00,0.00,0.00,0.00,0.040000,0.00,0.00,0.0
2,V5X,Oakridge,SE Oakridge,0.00,0.00,0.00,0.000000,0.00,0.0,0.02,...,0.00,0.02,0.02,0.02,0.00,0.00,0.000000,0.00,0.04,0.0
3,V5Z,Fairview,East Fairview,0.00,0.02,0.02,0.000000,0.04,0.0,0.04,...,0.02,0.04,0.00,0.00,0.02,0.02,0.000000,0.02,0.02,0.0
4,V5Z,South Cambie,South Cambie,0.00,0.00,0.00,0.021277,0.00,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.021277,0.00,0.00,0.0


4.8. Get the 10 most visited venue types in each area

In [26]:
mtl_top_venues = 10

indicators1 = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns1 = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns1 = []
for ind in np.arange(mtl_top_venues):
    try:
        freqColumns1.append('{}{} Most Common Venue'.format(ind+1, indicators1[ind]))
    except:
        freqColumns1.append('{}th Most Common Venue'.format(ind+1))
columns1 = areaColumns1+freqColumns1
# create a new dataframe
mtl_venues_sorted = pd.DataFrame(columns=columns1)
mtl_venues_sorted['PostalCode'] = Mtl_clust_venues_freq['PostalCode']
mtl_venues_sorted['Borough'] = Mtl_clust_venues_freq['Borough']
mtl_venues_sorted['Neighborhoods'] = Mtl_clust_venues_freq['Neighborhoods']

for ind in np.arange(Mtl_clust_venues_freq.shape[0]):
    row_categories1 = Mtl_clust_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted1 = row_categories1.sort_values(ascending=False)
    mtl_venues_sorted.iloc[ind, 3:] = row_categories_sorted1.index.values[0:mtl_top_venues]

mtl_venues_sorted.sort_values(freqColumns1, inplace=True)
mtl_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,H3S,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges North,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
20,H3T,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges Northeast,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
21,H3V,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges East,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
22,H3W,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges Southwest,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
25,H4A,Cote-des-Neiges-Notre-Dame-de-Grace,Notre-Dame-de-Grace Northeast,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
26,H4B,Cote-des-Neiges-Notre-Dame-de-Grace,Notre-Dame-de-Grace Southwest,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
5,H2V,Outremont,Outremont,Bakery,Park,Restaurant,Café,Italian Restaurant,Sandwich Place,Mediterranean Restaurant,Pharmacy,Ice Cream Shop,Food Service
16,H3K,Le Sud-Ouest,Pointe-Saint-Charles,Bar,Climbing Gym,Café,Gym,Cocktail Bar,Restaurant,Caribbean Restaurant,Museum,Film Studio,Indian Restaurant
0,H2H,Le Plateau-Mont-Royal,Plateau Mont-Royal North,Café,French Restaurant,Bakery,Tea Room,Restaurant,Portuguese Restaurant,Yoga Studio,Liquor Store,Jazz Club,Japanese Restaurant
1,H2J,Le Plateau-Mont-Royal,Plateau Mont-Royal North Central,Café,French Restaurant,Bakery,Tea Room,Restaurant,Portuguese Restaurant,Yoga Studio,Liquor Store,Jazz Club,Japanese Restaurant


In [27]:
van_top_venues = 10

indicators2 = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns2 = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns2 = []
for ind in np.arange(van_top_venues):
    try:
        freqColumns2.append('{}{} Most Common Venue'.format(ind+1, indicators2[ind]))
    except:
        freqColumns2.append('{}th Most Common Venue'.format(ind+1))
columns2 = areaColumns2+freqColumns2
# create a new dataframe
van_venues_sorted = pd.DataFrame(columns=columns2)
van_venues_sorted['PostalCode'] = Van_clust_venues_freq['PostalCode']
van_venues_sorted['Borough'] = Van_clust_venues_freq['Borough']
van_venues_sorted['Neighborhoods'] = Van_clust_venues_freq['Neighborhoods']

for ind in np.arange(Van_clust_venues_freq.shape[0]):
    row_categories2 = Van_clust_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted2 = row_categories2.sort_values(ascending=False)
    van_venues_sorted.iloc[ind, 3:] = row_categories_sorted2.index.values[0:van_top_venues]

van_venues_sorted.sort_values(freqColumns2, inplace=True)
van_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,V6R,West Point Grey,Jericho,Beach,Harbor / Marina,Park,Hostel,Fast Food Restaurant,Bus Stop,Café,Event Space,Garden,Disc Golf
1,V5X,Marpole,East Marpole,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
19,V6P,Marpole,West Marpole,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
0,V5W,Oakridge,NE Oakridge,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
2,V5X,Oakridge,SE Oakridge,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
13,V6M,Oakridge,NW Oakridge,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
20,V6P,Oakridge,SW Oakridge,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
12,V6M,Kerrisdale,NE Kerrisdale,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Bus Stop,Pharmacy,Sandwich Place,Tea Room,Pizza Place,Japanese Restaurant
17,V6N,Kerrisdale,West Kerrisdale,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Bus Stop,Pharmacy,Sandwich Place,Tea Room,Pizza Place,Japanese Restaurant
18,V6P,Kerrisdale,SE Kerrisdale,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Bus Stop,Pharmacy,Sandwich Place,Tea Room,Pizza Place,Japanese Restaurant


## 5) Neighborhood Clustering

5.1. Perform K-means clustering to see how the neighborhoods have been grouped.

In [28]:
kclusters = 3

Mtl_freq_clustering = Mtl_clust_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(Mtl_freq_clustering)

Mtl_clustered_df = Mtr_McGill_df
Mtl_clustered_df['Cluster'] = kmeans1.labels_

Mtl_clustered_df = Mtl_clustered_df.join(mtl_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
Mtl_clustered_df.sort_values(['Cluster'] + freqColumns1, inplace=True)
Mtl_clustered_df

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,69,H3S,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges North,45.476750,-73.614320,0,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
22,75,H3T,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges Northeast,45.476750,-73.614320,0,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
24,80,H3V,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges East,45.476750,-73.614320,0,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
26,84,H3W,Cote-des-Neiges-Notre-Dame-de-Grace,Cote-des-Neiges Southwest,45.476750,-73.614320,0,Bakery,Café,Sushi Restaurant,Restaurant,BBQ Joint,Pub,Korean Restaurant,Coffee Shop,Thai Restaurant,Liquor Store
23,79,H2V,Outremont,Outremont,45.514300,-73.609000,0,Bakery,Park,Restaurant,Café,Italian Restaurant,Sandwich Place,Mediterranean Restaurant,Pharmacy,Ice Cream Shop,Food Service
10,28,H2H,Le Plateau-Mont-Royal,Plateau Mont-Royal North,45.521667,-73.575556,0,Café,French Restaurant,Bakery,Tea Room,Restaurant,Portuguese Restaurant,Yoga Studio,Liquor Store,Jazz Club,Japanese Restaurant
21,74,H2T,Le Plateau-Mont-Royal,Plateau Mont-Royal West,45.521667,-73.575556,0,Café,French Restaurant,Bakery,Tea Room,Restaurant,Portuguese Restaurant,Yoga Studio,Liquor Store,Jazz Club,Japanese Restaurant
25,83,H2W,Le Plateau-Mont-Royal,Plateau Mont-Royal South Central,45.521667,-73.575556,0,Café,French Restaurant,Bakery,Tea Room,Restaurant,Portuguese Restaurant,Yoga Studio,Liquor Store,Jazz Club,Japanese Restaurant
14,38,H2K,Ville-Marie,Centre-Sud North,45.523565,-73.551870,0,Café,Gym,Gay Bar,Gastropub,Bakery,Breakfast Spot,Pizza Place,Concert Hall,Restaurant,Bar
28,93,H2Y,Ville-Marie,Old Montreal,45.501095,-73.556026,0,Café,Hotel,French Restaurant,History Museum,Restaurant,Spa,Portuguese Restaurant,Coffee Shop,Burger Joint,Seafood Restaurant


In [29]:
kclusters = 3

Van_freq_clustering = Van_clust_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(Van_freq_clustering)
Van_clustered_df = Vanc_UBC_df
Van_clustered_df['Cluster'] = kmeans2.labels_

Van_clustered_df = Van_clustered_df.join(van_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
Van_clustered_df.sort_values(['Cluster'] + freqColumns2, inplace=True)
Van_clustered_df

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,59,V6R,West Point Grey,West Point Grey,49.261000,-123.200100,0,Beach,Harbor / Marina,Park,Hostel,Fast Food Restaurant,Bus Stop,Café,Event Space,Garden,Disc Golf
22,60,V6R,West Point Grey,Jericho,49.273000,-123.203300,0,Beach,Harbor / Marina,Park,Hostel,Fast Food Restaurant,Bus Stop,Café,Event Space,Garden,Disc Golf
1,17,V5X,Oakridge,SE Oakridge,49.226100,-123.116600,0,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
2,18,V5X,Marpole,East Marpole,49.210700,-123.130200,0,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
18,55,V6P,Kerrisdale,SE Kerrisdale,49.234100,-123.155400,0,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
19,56,V6P,Marpole,West Marpole,49.210700,-123.130200,0,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
20,57,V6P,Oakridge,SW Oakridge,49.226100,-123.116600,0,Chinese Restaurant,Pizza Place,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Dessert Shop,Sandwich Place,Café,Liquor Store,Vietnamese Restaurant
0,15,V5W,Oakridge,NE Oakridge,49.226100,-123.116600,0,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
1,17,V5X,Oakridge,SE Oakridge,49.226100,-123.116600,0,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store
2,18,V5X,Marpole,East Marpole,49.210700,-123.130200,0,Coffee Shop,Bus Stop,Sandwich Place,Sporting Goods Shop,Sushi Restaurant,Fast Food Restaurant,Café,Tea Room,Park,Women's Store


5.2. Using the obtained data frames, display the clusters of neighborhoods within the Montreal's and Vancouver's maps.

In [30]:
# create map Montreal
map_mtl_clusters = folium.Map(location=[lam, lonm], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Mtl_clustered_df['Latitude'], Mtl_clustered_df['Longitude'], Mtl_clustered_df['PostalCode'], Mtl_clustered_df['Borough'], Mtl_clustered_df['Neighborhood'], Mtl_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mtl_clusters)
       
map_mtl_clusters

Upon observing the clusters and the associated information, it is possible to infer that:

1. Cluster 0 -> The venues part of this cluster are mainly Asian restaurants, bakeries, coffee shops, hotels and bars.
2. Cluster 1 -> It is the largest clusters and the venues part of this cluster are a wide variety of types restaurants, bakeries, coffee shops, tea shops, parks, banks, hotels and gyms. 
3. Cluster 2 -> The venues part of this cluster are mainly coffee shops, train stations, pharmacies, bakeries, parks, gyms, banks and some restaurants. Probably the neighborhoods part of this cluster, are living areas.

In [32]:
# create map
map_van_clusters = folium.Map(location=[lav, lonv], zoom_start=11.6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Van_clustered_df['Latitude'], Van_clustered_df['Longitude'], Van_clustered_df['PostalCode'], Van_clustered_df['Borough'], Van_clustered_df['Neighborhood'], Van_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_van_clusters)
       
map_van_clusters

Upon observing the clusters and the associated information, it is possible to infer that:

1. Cluster 0 -> It is the largest cluster and the venues part of this cluster are restaurants, coffee shops, parks, tea shops, markets, malls and sport centers.
2. Cluster 1 -> The venues part of this cluster are restaurants, coffee shops, parks, a golf course, gyms, tea shop, pharmacies and bars.
3. Cluster 2 -> The venues part of this cluster are coffee shops, gardens, parks, restaurants, malls, bakeries, bus stops, furniture stores, art/crafts stores and event spaces. The neighborhoods in this cluster, are probably mainly living areas.